In [2]:
!pip install -q --upgrade pip
!pip install -q "transformers>=4.30.0" "datasets" "accelerate>=0.21.0" "peft>=0.4.0" "bitsandbytes>=0.39.0" "sentencepiece" "safetensors"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.5 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
bigframes 2.12.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.31.0, but you have google-cloud-bigquery 3.25.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.1.0 which is incompatible.
libcugraph-cu12 25.6.0 requires libraft-cu12==25.6.*, but you have libraft-cu12 25.2.0 which is incompatible.
gradio 5.38.1 requires pyda

In [1]:
!pip install transformers accelerate torchinfo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 10.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.0 MB/s eta 0:00:00:00:0100:01
  Attempting un

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_NAME = "/kaggle/input/llama-3.2-3b-instruct/pytorch/default/1"   # ← change this to your model

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",      # loads layers automatically on available GPUs/CPU
    torch_dtype="auto"
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
from torchinfo import summary
import torch.nn as nn
summary(model, depth=9, col_names=["kernel_size", "num_params", "trainable"])

print("\nDetailed Linear Layers:\n" + "="*40)
for name, module in model.named_modules():
    if isinstance(module, nn.Linear):
        d, k = module.weight.shape
        print(f"{name:<70}  {d:>5} x {k:<5}  = {d*k:,} params")
        
print("\nTotal trainable parameters:", sum(p.numel() for p in model.parameters() if p.requires_grad))


Detailed Linear Layers:
model.layers.0.self_attn.q_proj                                          3072 x 3072   = 9,437,184 params
model.layers.0.self_attn.k_proj                                          1024 x 3072   = 3,145,728 params
model.layers.0.self_attn.v_proj                                          1024 x 3072   = 3,145,728 params
model.layers.0.self_attn.o_proj                                          3072 x 3072   = 9,437,184 params
model.layers.0.mlp.gate_proj                                             8192 x 3072   = 25,165,824 params
model.layers.0.mlp.up_proj                                               8192 x 3072   = 25,165,824 params
model.layers.0.mlp.down_proj                                             3072 x 8192   = 25,165,824 params
model.layers.1.self_attn.q_proj                                          3072 x 3072   = 9,437,184 params
model.layers.1.self_attn.k_proj                                          1024 x 3072   = 3,145,728 params
model.layers.1.sel

In [10]:
import torch
from prettytable import PrettyTable

# Get one transformer block (acts like one encoder/decoder layer)
block = model.model.layers[0]

print("=== TRANSFORMER BLOCK STRUCTURE ===")
print(block)

# Function to recursively list modules and parameter counts
def describe_module(module, prefix=""):
    rows = []
    total_params = 0
    for name, child in module.named_children():
        params = sum(p.numel() for p in child.parameters())
        total_params += params
        rows.append([prefix + name, child.__class__.__name__, f"{params:,}"])
        rows += describe_module(child, prefix + "  ")
    return rows

rows = describe_module(block)

# Pretty print the architecture
table = PrettyTable()
table.field_names = ["Layer Name", "Type", "Parameters"]
for r in rows:
    table.add_row(r)
print(table)


=== TRANSFORMER BLOCK STRUCTURE ===
LlamaDecoderLayer(
  (self_attn): LlamaAttention(
    (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
    (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
    (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
    (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
  )
  (mlp): LlamaMLP(
    (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
    (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
    (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
    (act_fn): SiLU()
  )
  (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
  (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
)
+--------------------------+----------------+------------+
|        Layer Name        |      Type      | Parameters |
+--------------------------+----------------+------------+
|        self_attn         | LlamaAttention | 25,165,824 |
|           q_

In [11]:
num_layers = len(model.model.layers)
print(f"Number of decoder layers: {num_layers}")

Number of decoder layers: 28
